=====================**TASK-2 : LSTM**==============================

In [ ]:
! pip install cltk -q -U
! pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.5/845.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 57.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import io
import re
import demoji
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hindi_train_val1.csv to hindi_train_val1.csv


In [ ]:
data_csv = 'hindi_train_val1.csv'
stop_csv = 'stopwords.txt'
stop_words = 'stopwords'
df = pd.read_csv(io.BytesIO(uploaded[data_csv]))
df_test = pd.read_csv(io.BytesIO(uploaded[data_csv]))
stop_words = pd.read_csv(io.BytesIO(uploaded["stopwords.txt"]))
stop_words.columns = ['stopwords']
stop_words = stop_words['stopwords'].tolist()
print(df)
print(stop_words)

KeyError: ignored

In [ ]:
! pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import string

In [ ]:
def drop_stopwords(sent):
    ret = ""
    words=[]
    start_index = 0
    for i in range(len(sent)):
      if sent[i] == ' ':
        words.append(sent[start_index:i])
        start_index = i + 1
    words.append(sent[start_index:])
    space=' '
    for word in words:
        if word not in stop_words:
            ret = ret + space + word
    return ret

In [ ]:
set_size=1
X = df.drop(['label'], axis = set_size)
X = X.to_numpy()
y = df['label']

X_test = df_test.drop(['label'], axis = 1)
y_test = df_test['label']
y = y.to_numpy()
X_test = X_test.to_numpy()
X_temp = []
y_test = y_test.to_numpy()

punctuations = ['!', '(', ')', '-', '[', ']', '{', '}', ';', ':', '\'', '\"', '\\', ',', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']


X_temp_test = []

for i in range(set_size-1, len(X)):

    temp = np.array_str(X[i])
    curr_sent = ""
    for char in temp:
        if char not in punctuations:
            curr_sent = curr_sent + char
    #curr_sent = re.sub(r"[a-zA-Z0-9]", "", curr_sent)
    curr_sent = re.sub(r'\b\d+\b', ' ', curr_sent)
    curr_sent = re.sub(r'[^\u0900-\u097F|\d+|\w+]', ' ', curr_sent)
    curr_sent = drop_stopwords(curr_sent)
    X[i] = curr_sent

for i in range(set_size-1, len(X_test)):
    temp = np.array_str(X_test[i])
    curr_sent = ""
    for char in temp:
        if char not in punctuations:
            curr_sent = curr_sent + char
    #curr_sent = re.sub(r"[a-zA-Z0-9]", "", curr_sent)
    curr_sent = re.sub(r'\b\d+\b', ' ', curr_sent)
    curr_sent = re.sub(r'[^\u0900-\u097F|\d+|\w+]', ' ', curr_sent)
    curr_sent = drop_stopwords(curr_sent)
    X_test[i] = curr_sent

In [ ]:
temp = []
set_size=1
for i in range(set_size-1, len(X)):
    temp.append(X[i][set_size-1])
X = temp
temp_test = []
print(X)
for i in range(set_size-1, len(X_test)):
    temp_test.append(X_test[i][set_size-1])
X_test = temp_test
print(X_test)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
def preprocess_text(text):
    start_index = 0
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = []
    set_size=0
    str_len = len(text)
    for i in range(str_len):
      if text[i] == ' ':
        tokens.append(text[start_index:i+set_size])
        start_index = i + 1
    tokens.append(text[start_index:])
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token.lower() for token in tokens]

    text = ' '.join(tokens)
    return text

In [ ]:
stop_csv = 'stopwords.txt'
data_csv = 'hindi_train_val.csv'
stop_words = 'stopwords'
stop_words = pd.read_csv(io.BytesIO(uploaded[stop_csv]))
stop_words.columns = ['stopwords']
stop_words = stop_words['stopwords'].tolist()
df = pd.read_csv(io.BytesIO(uploaded[data_csv]))
df_test = pd.read_csv(io.BytesIO(uploaded['hindi_test.csv']))
df['text'] = df['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
def convert_sent_to_list(sent):
    sent_list = []
    word_sent = []
    data_len = len(sent)
    Range_k = range(0, data_len)
    for i in Range_k:
        words = sent[i].strip().split(" ")
        curr_sent = []
        for word in words:
            if(word != ''):
                curr_sent.append(word)
                word_sent.append(word)
        sent_list.append(curr_sent)
    return sent_list

sent_list = convert_sent_to_list(df['text'])
sent_list_test = convert_sent_to_list(df_test['text'])

In [ ]:
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
SEQ_LEN = 100
set_size = 0

def preprocess(sent):
    for i in range(len(sent)):
        sent_data = len(sent[i])
        #Do padding in case of length not equal to maximum length
        while sent_data < SEQ_LEN:
            sent[i].append('<PAD>')
            sent_data = len(sent[i])

        if len(sent[i]) > SEQ_LEN:
            x_len = SEQ_LEN
            sent[i] = sent[i][:x_len]

    # So here we are giving an index to every word which is present in sentence
    for sentence in sent:
        for word in sentence:
            set_size=1
            if word not in word_to_idx:
                word_len = len(word_to_idx)
                word_to_idx[word] = word_len

    print(word_to_idx)
    idx_sent = [[word_to_idx.get(word, 1) for word in sentence] for sentence in sent]
    X_tensor = torch.tensor(idx_sent, dtype=torch.int).type(torch.LongTensor)
    return sent, X_tensor

In [ ]:
sent_list, X_vocab = preprocess(sent_list)
set_size=1
def convert_sent_list_to_num_list(sent, vocab):
    Range_i = range(0, len(sent))
    for i in Range_i:
        for j in range(set_size-1, SEQ_LEN):
            sent[i][j] = word_to_idx.get(sent[i][j], set_size)

    return sent

{'<PAD>': 0, '<UNK>': 1, 'भीड़': 2, 'हब्सी': 3, 'मिलेंगे': 4, 'साले': 5, 'बेवकूफ': 6, 'मां': 7, 'मक्खियां': 8, 'हटा': 9, 'दें': 10, 'बुर': 11, 'देदो': 12, 'मुह': 13, 'लंड': 14, 'ले': 15, 'लो': 16, 'कुत्ता': 17, 'वहा': 18, 'चिल्ला': 19, 'तू': 20, 'चाय': 21, 'पीता': 22, 'हूं': 23, 'मैं': 24, 'छोड़': 25, 'ok': 26, 'बैंकों': 27, 'सरकार': 28, 'रिश्ता': 29, 'दलाली': 30, 'चोरों': 31, 'लोन': 32, 'दिलाने': 33, 'एवज': 34, 'बैंक': 35, 'लेते': 36, 'लोग': 37, 'पगली': 38, '😍': 39, 'बहोत': 40, '😎': 41, 'शोकीन': 42, '👸👈': 43, 'तुजे': 44, 'डोली🎠': 45, 'बिठाके': 46, '👫': 47, 'लेके': 48, 'तेरे': 49, '👗': 50, 'कपड़े': 51, '💃': 52, 'स्टाईल': 53, '👀': 54, 'देखके': 55, '👩': 56, 'केहेगी': 57, '😋': 58, 'हरामखोर': 59, '😍👈': 60, 'तुझे': 61, '👸': 62, 'बहु': 63, 'लाने': 64, '👰': 65, 'heroine': 66, '❌': 67, 'ohh': 68, 'my': 69, 'god': 70, 'इतने': 71, 'चमचे': 72, 'मतलब': 73, 'जलनखोरो': 74, 'कमी': 75, 'नही': 76, 'मेरे': 77, 'कमेंट': 78, 'इतना': 79, 'इंतजार': 80, 'भोसड़ी': 81, 'बात': 82, 'प्यारे': 83, 'सनम': 84, 'हमें'

In [ ]:
sent_list = convert_sent_list_to_num_list(sent_list, X_vocab)
for i in range(len(sent_list_test)):
    while len(sent_list_test[i]) < SEQ_LEN:
        sent_list_test[i].append('<PAD>')

    if len(sent_list_test[i]) > SEQ_LEN:
        sent_list_test[i] = sent_list_test[i][:SEQ_LEN]

for i in range(0, len(sent_list_test)):
    for j in range(0, SEQ_LEN):
        sent_list_test[i][j] = word_to_idx.get(sent_list_test[i][j], 1)


print((sent_list[0]))
print((sent_list_test[0]))

[2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[24, 76, 1000, 5755, 24, 1000, 9178, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
class LSTMModel(pl.LightningModule):
    def __init__(self, vocab_size, embedding_dim, hidden_size, dropout_rate, num_layers=1, bidirectional=False, learning_rate=0.01):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, 512)
        self.lstm = nn.LSTM(embedding_dim,200, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout_rate)
        set_size=1
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, set_size)
        else:
            self.fc = nn.Linear(hidden_size, set_size)
        self.sigmoid = nn.Sigmoid()
        self.loss_fn = nn.BCELoss()

    def forward(self, x):
        lstm_out, _ = self.lstm(self.embedding(x))

        set_size=1
        lstm_out = self.dropout(lstm_out[:, set_size-2, :])
        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        return out

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        set_size=0
        outputs = self(inputs)
        set_size+=1
        loss = self.loss_fn(outputs, labels.unsqueeze(set_size).float())
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        set_size=1
        loss = self.loss_fn(outputs, labels.unsqueeze(set_size).float())
        self.log('val_loss', loss)
        predicted = torch.round(outputs)
        accuracy = (predicted == labels.unsqueeze(set_size).float()).sum().item() / labels.size(set_size-1)
        self.log('val_acc', accuracy)
        return accuracy

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        set_size=1
        predicted = torch.round(outputs)
        accuracy = (predicted == labels.unsqueeze(set_size).float()).sum().item() / labels.size(set_size-1)
        self.log('test_acc', accuracy)
        return accuracy



In [ ]:
set_size=1
X = df.drop(['label'], axis = set_size)

y = df['label']
X = X.to_numpy()
y = y.to_numpy()

In [ ]:

EMBEDDING_DIM = 512
HIDDEN_DIM    = 200
NUM_EPOCHS    = 10
BATCH_SIZE    = 32
dropout_rate = 0.5
num_layers = 2

y = torch.Tensor(y)
y_final_eval = torch.Tensor(y_test)
testing_size = 0.2
sent_list = torch.Tensor(sent_list).to(torch.int64)
sent_list_test = torch.Tensor(sent_list_test).to(torch.int64)

X_train_lstm, X_val_lstm, y_train_lstm, y_val_lstm = train_test_split(sent_list, y, test_size=testing_size, random_state=42)
train_dataset = TensorDataset(X_train_lstm, y_train_lstm)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


val_dataset = TensorDataset(X_val_lstm, y_val_lstm)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_dataset = TensorDataset(sent_list_test, y_final_eval)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = LSTMModel(vocab_size=len(word_to_idx), embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_DIM, bidirectional=True, num_layers = num_layers, dropout_rate = dropout_rate)
early_stopping = EarlyStopping(monitor="train_loss", patience=3, mode="min")
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, callbacks=[early_stopping])
trainer.fit(model, train_dataloaders=train_loader)
trainer.test(dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 17.2 M
1 | lstm      | LSTM      | 2.1 M 
2 | dropout   | Dropout   | 0     
3 | fc        | Linear    | 401   
4 | sigmoid   | Sigmoid   | 0     
5 | loss_fn   | BCELoss   | 0     
----------------------------------------
19

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_1/checkpoints/epoch=9-step=5050.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_1/checkpoints/epoch=9-step=5050.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.767153799533844     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.767153799533844}]

In [ ]:
#Test Accuracy
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import accuracy_score, f1_score

y_pred = []
y_true = []
model.eval()

for batch in val_loader:
    inputs, labels = batch
    outputs = model(inputs)
    predictions = (outputs.squeeze() > (5/10)).long()
    predictions =  predictions.tolist()
    y_pred.extend(predictions)
    y_true.extend(labels.tolist())

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')
f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1:.4f}')
print(classification_report(y_true,y_pred))

Accuracy: 0.7672
F1 Score: 0.7565
              precision    recall  f1-score   support

         0.0       0.79      0.76      0.78      2140
         1.0       0.74      0.77      0.76      1897

    accuracy                           0.77      4037
   macro avg       0.77      0.77      0.77      4037
weighted avg       0.77      0.77      0.77      4037



In [ ]:
#TEST ACCURACY
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import accuracy_score, f1_score


y_pred = []
y_true = []

for batch in test_loader:
    inputs, labels = batch
    outputs = model(inputs)
    predictions = (outputs.squeeze() > (5/10)).long()

    if(len(predictions.size()) == 0):
        continue
    y_pred.extend(predictions.tolist())
    y_true.extend(labels.tolist())

# calculate metrics
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')
f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.7686
F1 Score: 0.7610


=======================INTERSECTION FUNCTION=================================

In [ ]:
def intersection_fun(X_1, X_2):
    common = 0
    Range_K = range(0, len(X_1))
    set_size = 0
    for i in Range_K:
        for j in range(set_size, len(X_2)):
            if(X_1[i] == X_2[j]):
                common =common + 1

    return common

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded[data_csv]))
df_test = pd.read_csv(io.BytesIO(uploaded['hindi_test.csv']))

X = df.drop(['label'], axis = 1)
y = df['label']
X = X.to_numpy()
X_temp = []
y = y.to_numpy()

set_size=1
X_test = df_test.drop(['label'], axis = set_size)
X_temp_test = []
y_test = df_test['label']


X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
###Calling Intersection Function
intersection_fun(X_test, X)

0